In [29]:
import pandas as pd
import qiime2 as q2
from biom import Table
import numpy as np
import pandas as pd
from qiime2.plugins.phylogeny.methods import filter_table
import time

from birdman import NegativeBinomialLME


In [30]:
table_metag = q2.Artifact.load('../data/metaG/metaG-matched-table.qza')
metadata_metag = q2.Metadata.load('../data/metaG/metaG-matched-metadata.qza').to_dataframe()
tree_metag = q2.Artifact.load('../data/metaG/wol_tree.qza')
table_metag = filter_table(table_metag, tree_metag).filtered_table
bt_table_metag = table_metag.view(Table)

# filter and match the table
n_features, n_samples = bt_table_metag.shape
min_feature_frequency = 10
min_feature_count = 0
min_sample_count = 500000
def frequency_filter(val, id_, md):
    return (np.sum(val > 0) / n_samples) > (min_feature_frequency / 100)
def observation_filter(val, id_, md):
        return sum(val) > min_feature_count
bt_table_metag = bt_table_metag.filter(bt_table_metag.ids()[(bt_table_metag.sum('sample') > min_sample_count)])
bt_table_metag = bt_table_metag.filter(observation_filter, axis='observation')
bt_table_metag = bt_table_metag.filter(frequency_filter, axis='observation')
metadata_metag = metadata_metag.reindex(bt_table_metag.ids())
bt_table_metag


3429 x 86 <class 'biom.table.Table'> with 165335 nonzero entries (56% dense)

In [31]:
formula = "first_diet+diet_nocross+timepoint_encoded+cog"
subj = "host_subject_id"

nb = NegativeBinomialLME(
    table=bt_table_metag,
    formula=formula,
    group_var=subj,
    metadata=metadata_metag,
    num_iter=500,
)


start = time.time()
nb.compile_model()
end = time.time()
print(end - start)

start = time.time()
nb.fit_model()
end = time.time()
print(end - start)



INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /Users/cmartino/miniconda3/envs/qiime2-2021.2.songbird.mmvec/lib/python3.6/site-packages/birdman/templates/negative_binomial_lme
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:start chain 4


0.0020918846130371094


INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 2
KeyboardInterrupt

INFO:cmdstanpy:finish chain 1



In [ ]:
from birdman.transform import posterior_alr_to_clr

inference = nb.to_inference_object()
inference.posterior = posterior_alr_to_clr(inference.posterior)

In [ ]:
import birdman.visualization as viz

ax = viz.plot_parameter_estimates(
    inference,
    parameter="beta",
    coord={"covariate": "diet[T.DIO]"},
)